<a href="https://colab.research.google.com/github/Jashwanth63/Custom-Model-for-classification-of-Image-Data-without-using-Convolutional-Neural-Networks/blob/main/Deep_Learning_Assignment_2022_Jashwanth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>